In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import re
import string

#matplotlib imports are used to plot confusion matrices for the classifiers
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 

# Process accented characters
import unicodedata

# NLTK
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

# Machine learning
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Import different metrics to evaluate the classifiers
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 

from sklearn.utils import resample

#import time function from time module to track the training duration
from time import time

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Boss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Boss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Connect to database

In [2]:
# SQL Alchemy
from sqlalchemy import create_engine

In [3]:
# Create engine and connection
engine = create_engine("sqlite:///../db.sqlite")

In [4]:
# Check for tables
engine.table_names()

['cuisine_ingredients']

In [5]:
# Read in the csv file
df = pd.read_sql_query('SELECT * FROM cuisine_ingredients', con=engine)

In [6]:
# Overview of the data set
df.sample(10, random_state=2)

,id,cuisine,recipe,full_ingredients
3727,3728,Italian,Spicy fregola with scallops and crispy kale,"['olive oil, for shallow and deep frying', '1 ..."
1923,1924,Chinese,Chunky bacon and cucumber salad,"['1 tbsp groundnut oil', '6 long dried chillie..."
3552,3553,Italian,Polenta pork,"['2 tbsp sunflower oil ', '12 rashers thick ri..."
1146,1147,British,Oat fig stuffing,"['140g/5oz butter', '100g/3½oz jumbo oats', '1..."
3195,3196,Italian,Bolognese with tagliatelle,"['250g/9oz 00 flour', '3-4 medium free-range e..."
1643,1644,British,Textured potato salad,"['675g/1½lb floury potatoes, peeled and cut in..."
3054,3055,Indian,Tandoori lamb wrap,"['150ml/5fl oz Greek-style yoghurt ', '½ small..."
842,843,British,Gammon and pease pudding,"['300g/10oz dried yellow split peas', '50g/2oz..."
505,506,British,Braised shin of beef with parsnip purée,"['4kg/9lb beef shin, on the bone', 'sea salt a..."
2224,2225,French,Mary Berry's celeriac remoulade,"['650g/1lb 7oz celeriac, peeled and sliced int..."


In [7]:
# Get a list of unique cuisines in the dataset
cuisine_list = df.cuisine.unique().tolist()
cuisine_list

['African',
 'American',
 'British',
 'Caribbean',
 'Chinese',
 'East European',
 'French',
 'Greek',
 'Indian',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Mexican',
 'Nordic',
 'North African',
 'Pakistani',
 'Portuguese',
 'South American',
 'Spanish',
 'Thai and South-East Asian',
 'Turkish and Middle Eastern']

## Define text pre-processing function

In [8]:
# Create a function to remove accented characters
def remove_accented_chars(matchobj):
    text = matchobj.group()
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

I'm combining the text cleaning and pre-processing steps as explored and explained in the 00_EDA file.

In [9]:
# Create a function to clean ingredient text
def clean(doc):
    doc = doc.str.lower()
    doc = doc.str.replace(r'\w*[\d¼½¾⅓⅔⅛⅜⅝]\w*', '')
    doc = doc.str.translate(str.maketrans('', '', string.punctuation))
    doc = doc.str.replace(r'[£×–‘’“”⁄]', '')
#     doc = doc.apply(lambda x: re.sub(r'[âãäçèéêîïñóôûüōưấớ]', remove_accented_chars, x))
    doc = doc.apply(lambda x: word_tokenize(x))
    doc = doc.apply(lambda x: [word for word in x if not word in stop_words_nltk])
    doc = doc.apply(lambda x: [stemmer.stem(word) for word in x])
    processed_doc = doc.apply(lambda x: ' '.join([word for word in x]))

    return processed_doc

In [10]:
# Add a new column to the dataframe with the cleaned text
df["ingredients_processed"] = clean(df.full_ingredients)

In [11]:
# Overview of the dataset
df.sample(10, random_state=2)

,id,cuisine,recipe,full_ingredients,ingredients_processed
3727,3728,Italian,Spicy fregola with scallops and crispy kale,"['olive oil, for shallow and deep frying', '1 ...",oliv oil shallow deep fri shallot fine chop ga...
1923,1924,Chinese,Chunky bacon and cucumber salad,"['1 tbsp groundnut oil', '6 long dried chillie...",tbsp groundnut oil long dri chilli tsp sichuan...
3552,3553,Italian,Polenta pork,"['2 tbsp sunflower oil ', '12 rashers thick ri...",tbsp sunflow oil rasher thick rindless smoke s...
1146,1147,British,Oat fig stuffing,"['140g/5oz butter', '100g/3½oz jumbo oats', '1...",butter jumbo oat roll oat red onion roughli ch...
3195,3196,Italian,Bolognese with tagliatelle,"['250g/9oz 00 flour', '3-4 medium free-range e...",flour medium freerang egg prefer bright yellow...
1643,1644,British,Textured potato salad,"['675g/1½lb floury potatoes, peeled and cut in...",flouri potato peel cut cube rasher smoke strea...
3054,3055,Indian,Tandoori lamb wrap,"['150ml/5fl oz Greek-style yoghurt ', '½ small...",oz greekstyl yoghurt small onion roughli chop ...
842,843,British,Gammon and pease pudding,"['300g/10oz dried yellow split peas', '50g/2oz...",dri yellow split pea butter onion roughli chop...
505,506,British,Braised shin of beef with parsnip purée,"['4kg/9lb beef shin, on the bone', 'sea salt a...",beef shin bone sea salt freshli ground black p...
2224,2225,French,Mary Berry's celeriac remoulade,"['650g/1lb 7oz celeriac, peeled and sliced int...",celeriac peel slice thin matchstick see tip le...


## Modeling

### Split the data into training and test sets (75% train and 25% test)

In [12]:
# The column contains textual data to extract features from.
X = df.ingredients_processed

# The column we're learning to predict.
y = df.cuisine
print(X.shape, y.shape)

(4724,) (4724,)


In [13]:
# Split X and y into training and testing sets. By default, it splits 75% training and 25% test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3543,) (3543,)
(1181,) (1181,)


### Resampling

In [14]:
# Concatenate our training data back together
X_y_train = pd.concat([X_train, y_train], axis=1)
X_y_train.shape

(3543, 2)

In [15]:
# Overview of the training set
X_y_train.head()

,ingredients_processed,cuisine
1948,fresh white fish fillet sole plaic cornflour d...,Chinese
921,tbsp oliv oil goodqual pork sausag shallot fin...,British
4215,tbsp veget oil onion fine chop garlic clove fi...,Pakistani
2601,x sirloin steak salt freshli ground black pepp...,French
3770,type pasta flour avail supermarket bake sectio...,Italian


In [16]:
# Separate minority and majority classes
british_cuisines_df = X_y_train[X_y_train.cuisine=="British"]
other_cuisines_df = X_y_train[X_y_train.cuisine!="British"]
other_cuisines_df.head()

,ingredients_processed,cuisine
1948,fresh white fish fillet sole plaic cornflour d...,Chinese
4215,tbsp veget oil onion fine chop garlic clove fi...,Pakistani
2601,x sirloin steak salt freshli ground black pepp...,French
3770,type pasta flour avail supermarket bake sectio...,Italian
3737,tagliatel broccoli cut small floret hand pea f...,Italian


In [17]:
# Get a list of minority cuisines
other_cuisines = other_cuisines_df.cuisine.unique().tolist()
other_cuisines

['Chinese',
 'Pakistani',
 'French',
 'Italian',
 'Japanese',
 'Indian',
 'Mexican',
 'American',
 'Nordic',
 'Turkish and Middle Eastern',
 'Caribbean',
 'Thai and South-East Asian',
 'North African',
 'East European',
 'Irish',
 'Portuguese',
 'Greek',
 'South American',
 'Spanish',
 'Korean',
 'African']

In [18]:
other_cuisines_upsampled = list()

# Upsample the minorities

for cuisine in other_cuisines:
    cuisine_df = X_y_train[X_y_train.cuisine==cuisine]
    cuisine_upsampled = resample(cuisine_df,
                                 replace=True, # sample with replacement
                                 n_samples=len(british_cuisines_df), # match number of recipes in British cuisine
                                 random_state=1)
    other_cuisines_upsampled.append(cuisine_upsampled)

In [19]:
# Create a new resampled data set for minority cuisines
other_cuisines_upsampled = pd.concat(other_cuisines_upsampled)
other_cuisines_upsampled.head()

,ingredients_processed,cuisine
1909,tbsp veget oil plu extra shallow fri firm tofu...,Chinese
2041,firm white fish fillet cod sole skin whole fis...,Chinese
2019,tbsp shaox rice wine dri sherri tbsp light soy...,Chinese
1980,x lobster tsp sesam oil fresh root ginger peel...,Chinese
1999,pork minc ginger grate garlic grate spring oni...,Chinese


In [20]:
# Combine the majority and the upsampled minority
upsampled = pd.concat([british_cuisines_df, other_cuisines_upsampled])

In [21]:
# check new class counts
upsampled.cuisine.value_counts()

Japanese                      1098
British                       1098
Pakistani                     1098
Irish                         1098
French                        1098
American                      1098
Chinese                       1098
Mexican                       1098
South American                1098
Caribbean                     1098
Portuguese                    1098
Greek                         1098
Turkish and Middle Eastern    1098
Spanish                       1098
North African                 1098
Thai and South-East Asian     1098
Korean                        1098
East European                 1098
Nordic                        1098
African                       1098
Indian                        1098
Italian                       1098
Name: cuisine, dtype: int64

In [22]:
# Overview of the upsampled dataset
upsampled.sample(10, random_state=2)

,ingredients_processed,cuisine
4133,oliv oil fri larg chicken thigh onion fine cho...,North African
3773,flour freerang italian yellow golden egg durum...,Italian
2592,salt butter room temperatur caster sugar star ...,French
4159,small chicken joint piec tbsp oliv oil preserv...,North African
4204,tbsp ghee clarifi butter onion fine chop fresh...,Pakistani
3943,fillet beef garlic clove fresh root ginger sli...,Korean
4287,whole king prawn around medium oz oliv oil gar...,Spanish
4046,tbsp oliv oil plu extra greas beef minc red on...,Mexican
2145,salmon fillet skin longgrain rice butter plu e...,East European
4413,duck breast skin oz veget oil oz dark soy sauc...,Thai and South-East Asian


### Feature engineering using TF-IDF

In bags of words and bags of n-grams approaches, all words in a corpus are treated equally important. TF-IDF, meanwhile, emphasizes that some words in a document are more important than others. For the current classification problem, I find that TF-IDF would suit the best.

In [23]:
X_train_new = upsampled.ingredients_processed
y_train_new = upsampled.cuisine

In [27]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
tfidf_count_occurs = tfidf_vec.fit_transform(X_train_new)
tfidf_count_occur_df = pd.DataFrame(
    (count, word) for word, count in zip(
    tfidf_count_occurs.toarray().tolist()[0],   
    tfidf_vec.get_feature_names()))
tfidf_count_occur_df.columns = ['Word', 'Count']
tfidf_count_occur_df.sort_values('Count', ascending=False, inplace=True)
tfidf_count_occur_df.head()

MemoryError: 

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X_train_new = upsampled.ingredients_processed
y_train_new = upsampled.cuisine

In [ ]:
# Vectorize train and test data
X_train_transformed = tfidf.fit_transform(X_train_new)
print(X_train_transformed.shape)

### Train the model

In [ ]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
# Train the classifier and time the training step
%time model.fit(X_train_transformed, y_train_new)

In [ ]:
# Make class predictions for X_test_transformed
X_test_transformed = tfidf.transform(X_test)
y_predicted = model.predict(X_test_transformed)
y_predicted

 ### Evaluate the model using accuracy, confusion matrix, and classification report

In [ ]:
# Use a Dummy Classifer to classify cuisines without even training a "real" model
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier().fit(X_train_transformed, y_train_new)
dummy_pred = dummy.predict(X_test_transformed)

# Check accuracy on test set
print('Dummy Classifer Test Accuracy: ', accuracy_score(y_test, dummy_pred))

In [ ]:
# Print accuracy on test set:
print("Test Accuracy: ", accuracy_score(y_test, y_predicted))

In [ ]:
# Print accuracy on train set:
print("Train Accuracy: ", model.score(X_train_transformed, y_train_new))

> Accuracy is not the best metric to evaluate imbalanced testing datasets as it can be very misleading. I'll use confusion matrix and classification report for further understanding about the model performance.

In [ ]:
# Function to plot confusion matrix. 
# Ref: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize=15)
    plt.xlabel('Predicted label',fontsize=15)

In [ ]:
# print the confusion matrix
cnf_matrix = confusion_matrix(y_test, y_predicted)
plt.figure(figsize=(16, 12))
plot_confusion_matrix(cnf_matrix, classes=cuisine_list, normalize=True,
                      title='Confusion matrix with all features')

At this point, we can notice that the classifier is doing poorly with identifying xxx, while it is doing well with xxx.

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted,
                            target_names=cuisine_list))

So, how do we choose whats the best? If we look at overall accuracy alone, we should be choosing the very first classifier in this notebook. However, that is also doing poorly with identifying "relevant" articles. If we choose purely based on how good it is doing with "relevant" category, we should choose the second one we built. If we choose purely based on how good it is doing with "irrelevant" category, surely, nothing beats not building any classifier and just calling everything irrelevant! So, what to choose as the best among these depends on what we are looking for in our usecase!

### Evaluate the model using k-fold cross validation

In [ ]:
new_tfidf = TfidfVectorizer()

In [ ]:
# The column contains textual data to extract features from.
X = df.ingredients_processed
X_transformed = new_tfidf.fit_transform(X)

# The column we're learning to predict.
y = df.cuisine 
print(X.shape, y.shape)

In [ ]:
# Create 10 folds
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
# Train and evaluate multiple models using kfolds
results = cross_val_score(model, X_transformed, y, cv=kfold)
print(results)
print("Accuracy Score Mean:", results.mean())
print("Accuracy Score Std:", results.std())

### Hyperparameter Tuning

Use GridSearchCV to tune the model's parameters

In [ ]:
# # Create the GridSearch estimator along with a parameter object containing the values to adjust
# from sklearn.model_selection import GridSearchCV
# param_grid = {'kernel': ["linear", "poly", "rbf", "sigmoid"]}
# grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch

In [ ]:
# print(grid2.best_params_)
# print(grid2.best_score_)

### Test model on one example

In [ ]:
text = "butter, for greasing \
        400ml/14fl oz full-fat milk \
        50g/1¾oz fresh white breadcrumbs \
        4 large free-range eggs, beaten \
        1 tbsp olive oil \
        1kg/2lb 4oz lamb mince \
        2 onions, finely chopped \
        1 bay leaf \
        2 garlic cloves, crushed \
        2 tbsp madras curry paste \
        250ml/9fl oz lamb stock \
        2 tbsp Worcestershire sauce \
        2 tbsp mango chutney \
        50g/1¾oz raisins \
        50g/1¾oz dried apricots, chopped \
        1 tbsp cider vinegar"

In [ ]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\w*[\d¼½¾⅓⅔⅛⅜⅝]\w*', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[£×–‘’“”⁄]', '', text)
    words = word_tokenize(text)
    words_no_stopwords = [word for word in words if not word in stop_words_nltk]
    stemmed_words = [stemmer.stem(word) for word in words_no_stopwords]
    processed_text = ' '.join([word for word in stemmed_words])
    
    return processed_text

In [ ]:
X_cleaned = pd.Series([preprocess(text)])
X_cleaned

In [ ]:
result = model.predict(tfidf.transform(X_cleaned))
result